In [1]:
!pip install transformers datasets accelerate loralib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments, DistilBertTokenizer
from datasets import load_dataset
from transformers import get_scheduler
import loralib as lora
import time

from datasets import concatenate_datasets

from sklearn.metrics import precision_recall_fscore_support
import numpy as np
import pandas as pd

# Create dataset

In [3]:
# Load the IMDB dataset
dataset = load_dataset("imdb")

# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize the IMDB dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
len(train_dataset)

25000

In [8]:

eval_dataset = test_dataset.shuffle(seed=42).select(range(len(test_dataset)//2))

test_dataset = test_dataset.shuffle(seed=42).select(  range(  len(test_dataset)//2+1, len(test_dataset) ) )

In [11]:
#check eval_dataset and test_dataset is balanced
eval_pos = 0
eval_neg = 0
for i in range(len(eval_dataset)):
    if eval_dataset[i]['label'] == 1:
        eval_pos += 1
    else:
        eval_neg += 1
print(eval_pos, eval_neg)

6233 6267


In [13]:
#check eval_dataset and test_dataset is balanced
test_pos = 0
test_neg = 0
for i in range(len(test_dataset)):
    if test_dataset[i]['label'] == 1:
        test_pos += 1
    else:
        test_neg += 1
print(test_pos, test_neg)

6266 6233


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
import torch
torch.cuda.empty_cache()


In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="binary", zero_division=1)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }


In [ ]:
5e7abf784a3f7805725b2a241d1a5b31d222be1d

# Original model

In [15]:
org_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
org_model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [17]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    # Convert lists to tensors directly and use padding
    inputs = {key: torch.tensor([example[key] for example in batch]) for key in tokenizer.model_input_names}
    labels = torch.tensor([example["label"] for example in batch])
    return inputs, labels

# Set up DataLoader with the new collate function
dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=collate_fn)


In [18]:
dataloader

In [19]:
# Move model to GPU if available

# Store predictions and true labels
predictions = []
true_labels = []

# Predict on each batch
for batch in dataloader:
    inputs, labels = batch
    inputs = {k: v.to(device) for k, v in inputs.items()}
    labels = labels.to(device)

    with torch.no_grad():
        outputs = org_model(**inputs)
        logits = outputs.logits
        predicted_classes = torch.argmax(logits, dim=-1)

        # Append predictions and labels to lists
        predictions.extend(predicted_classes.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())


In [20]:
type(predictions)

list

In [21]:
sum(true_labels)

6266

In [22]:
from sklearn.metrics import precision_recall_fscore_support

# Calculate precision, recall, and F1-score
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary', zero_division=1)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")


Precision: 0.6756756756756757
Recall: 0.003989786147462496
F1-score: 0.007932730445819451


# Supervised fine tune

In [23]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import time

# Load the DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    load_best_model_at_end=True,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Fine-tune the model
start_time = time.time()
trainer.train()
full_finetune_time = time.time() - start_time
full_finetune_params = sum(p.numel() for p in model.parameters() if p.requires_grad)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.289900,0.266383,0.949982,0.859297,0.902367
2,0.193700,0.309994,0.930352,0.897962,0.913871
3,0.102700,0.374237,0.945590,0.897802,0.921076
4,0.027000,0.540267,0.937802,0.902294,0.919706


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.289900,0.266383,0.949982,0.859297,0.902367
2,0.193700,0.309994,0.930352,0.897962,0.913871
3,0.102700,0.374237,0.945590,0.897802,0.921076
4,0.027000,0.540267,0.937802,0.902294,0.919706
5,0.011100,0.566220,0.926616,0.917696,0.922134


In [24]:
evaluation_results = trainer.evaluate()
print(evaluation_results)

{'eval_loss': 0.26638278365135193, 'eval_precision': 0.9499822632139057, 'eval_recall': 0.8592972886250602, 'eval_f1': 0.9023671131328448, 'eval_runtime': 176.2461, 'eval_samples_per_second': 70.924, 'eval_steps_per_second': 8.868, 'epoch': 5.0}


In [34]:
from transformers import T5ForConditionalGeneration, Trainer

# Define the save path
save_path = "content/distilbert"

# Save the fine-tuned model
trainer.save_model(save_path)  # Saves the model, config, and optimizer state (if any)




In [37]:
save_path = "/content/drive/MyDrive/5300_project_model_checkpoint/distilbert_sft"  # Replace with your desired path in Google Drive

# Save the fine-tuned model
trainer.save_model(save_path)

print(f"Model and tokenizer saved to {save_path}")


Model and tokenizer saved to /content/drive/MyDrive/5300_project_model_checkpoint/distilbert_sft


In [40]:
import shutil

# Path to the saved model folder
save_path = "content/distilbert"

# Create a ZIP file
shutil.make_archive(save_path, 'zip', save_path)

print(f"Folder compressed into {save_path}.zip")


Folder compressed into content/distilbert.zip


In [38]:
model = T5ForConditionalGeneration.from_pretrained(save_path)

You are using a model of type distilbert to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at /content/drive/MyDrive/5300_project_model_checkpoint/distilbert_sft and are newly initialized: ['decoder.block.0.layer.0.SelfAttention.k.weight', 'decoder.block.0.layer.0.SelfAttention.o.weight', 'decoder.block.0.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.0.layer.0.SelfAttention.v.weight', 'decoder.block.0.layer.0.layer_norm.weight', 'decoder.block.0.layer.1.EncDecAttention.k.weight', 'decoder.block.0.layer.1.EncDecAttention.o.weight', 'decoder.block.0.layer.1.EncDecAttention.q.weight', 'decoder.block.0.layer.1.EncDecAttention.v.weight', 'decoder.block.0.layer.1.layer_norm.weight', 'decoder.block.0.layer.2.DenseReluDense.wi.weight', 'decoder.block.0.layer.2.DenseReluDe

In [ ]:
full_finetune_time

1059.4597222805023

In [ ]:
full_finetune_params

66955010

In [27]:
# Move model to GPU if available

# Store predictions and true labels
predictions = []
true_labels = []

# Predict on each batch
for batch in dataloader:
    inputs, labels = batch
    inputs = {k: v.to(device) for k, v in inputs.items()}
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_classes = torch.argmax(logits, dim=-1)

        # Append predictions and labels to lists
        predictions.extend(predicted_classes.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())


In [28]:
from sklearn.metrics import precision_recall_fscore_support

# Calculate precision, recall, and F1-score
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='binary', zero_division=1)
print ("after full supervised fine tune")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")


after full supervised fine tune
Precision: 0.9517131191194745
Recall: 0.8555697414618576
F1-score: 0.9010841247163627
